In [1]:
import pickle
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
tr = pickle.load(open('tr.pkl','rb'))
te = pickle.load(open('te.pkl','rb'))
ui = pickle.load(open('ui.pkl','rb'))
ul = pickle.load(open('ul.pkl','rb'))

In [3]:
d = pd.concat([tr,te],axis = 0).reset_index(drop = True)
d = d.merge(ul,on = ['user_id'],how = 'left')
d

,user_id,merchant_id,label,prob,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,34176,3906,0.0,NaN,790010,1467,4231,3588.0,1026,0
1,34176,3906,0.0,NaN,231901,662,1945,2758.0,818,0
2,34176,3906,0.0,NaN,478025,662,3700,2204.0,818,0
3,34176,3906,0.0,NaN,100290,662,2179,6740.0,818,0
4,34176,3906,0.0,NaN,655967,420,1875,2714.0,1110,0
...,...,...,...,...,...,...,...,...,...,...
74592158,32639,3319,NaN,NaN,18918,1228,2200,4936.0,925,0
74592159,32639,3319,NaN,NaN,394570,1413,1065,6376.0,523,0
74592160,32639,3319,NaN,NaN,394570,1413,1065,4468.0,523,2
74592161,32639,3319,NaN,NaN,394570,1413,1065,6376.0,523,0


In [4]:
d = d[d['merchant_id'] == d['seller_id']] \
.reset_index(drop = True)
d

,user_id,merchant_id,label,prob,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,34176,3906,0.0,NaN,757713,821,3906,6268.0,1110,0
1,34176,3906,0.0,NaN,757713,821,3906,6268.0,1110,0
2,34176,3906,0.0,NaN,757713,821,3906,6268.0,1110,0
3,34176,3906,0.0,NaN,718096,1142,3906,6268.0,1031,3
4,34176,3906,0.0,NaN,757713,821,3906,6268.0,1031,3
...,...,...,...,...,...,...,...,...,...,...
5662510,32639,3319,NaN,NaN,179481,898,3319,5508.0,1111,0
5662511,32639,3319,NaN,NaN,179481,898,3319,5508.0,1111,0
5662512,32639,3319,NaN,NaN,179481,898,3319,5508.0,1111,0
5662513,32639,3319,NaN,NaN,179481,898,3319,5508.0,1111,0


In [5]:
del tr,te,ui
import gc
gc.collect()

655

In [6]:
ul = ul.sort_values(['user_id','time_stamp'])
ul

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
23288890,1,181459,276,2245,4752.0,1009,0
23288891,1,779078,276,2245,4752.0,1009,0
23288892,1,779078,276,2245,4752.0,1009,0
23288893,1,452837,276,2245,4752.0,1009,0
23288894,1,543397,276,2245,4752.0,1009,0
...,...,...,...,...,...,...,...
13710705,424170,416729,602,3736,3124.0,1111,0
13710706,424170,424015,761,525,5444.0,1111,0
13710707,424170,802762,602,3736,3124.0,1111,0
13710714,424170,795753,656,4268,1642.0,1111,2


In [7]:
ul.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
23288890,1,181459,276,2245,4752.0,1009,0
23288891,1,779078,276,2245,4752.0,1009,0
23288892,1,779078,276,2245,4752.0,1009,0
23288893,1,452837,276,2245,4752.0,1009,0
23288894,1,543397,276,2245,4752.0,1009,0


In [8]:
temp = pd.get_dummies(ul['action_type'],
                      prefix="action_type")

In [9]:
temp = pd.concat([ul['user_id'],temp],axis = 1).groupby(by = ['user_id'],as_index = False).sum()

In [10]:
temp

,user_id,action_type_0,action_type_1,action_type_2,action_type_3
0,1,27.0,0.0,6.0,0.0
1,2,47.0,0.0,14.0,2.0
2,3,63.0,0.0,4.0,1.0
3,4,49.0,0.0,1.0,0.0
4,5,150.0,0.0,13.0,10.0
...,...,...,...,...,...
424165,424166,79.0,0.0,11.0,0.0
424166,424167,28.0,0.0,6.0,1.0
424167,424168,216.0,0.0,6.0,1.0
424168,424169,277.0,0.0,17.0,3.0


In [11]:
temp.loc[:,'action_type_0']

0          27.0
1          47.0
2          63.0
3          49.0
4         150.0
          ...  
424165     79.0
424166     28.0
424167    216.0
424168    277.0
424169     39.0
Name: action_type_0, Length: 424170, dtype: float64

In [13]:
action_type = temp
action_type

,user_id,action_type_0,action_type_1,action_type_2,action_type_3
0,1,27.0,0.0,6.0,0.0
1,2,47.0,0.0,14.0,2.0
2,3,63.0,0.0,4.0,1.0
3,4,49.0,0.0,1.0,0.0
4,5,150.0,0.0,13.0,10.0
...,...,...,...,...,...
424165,424166,79.0,0.0,11.0,0.0
424166,424167,28.0,0.0,6.0,1.0
424167,424168,216.0,0.0,6.0,1.0
424168,424169,277.0,0.0,17.0,3.0


In [14]:
action_type = action_type.groupby(by = ['user_id'],
                                  as_index = False).sum()
action_type['action_sum'] = action_type.iloc[:,1:] \
.sum(axis = 1)

In [15]:
action_type.head()

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_sum
0,1,27.0,0.0,6.0,0.0,33.0
1,2,47.0,0.0,14.0,2.0,63.0
2,3,63.0,0.0,4.0,1.0,68.0
3,4,49.0,0.0,1.0,0.0,50.0
4,5,150.0,0.0,13.0,10.0,173.0


In [16]:
action_all = action_type['action_sum'].sum()
action_type['action_ratio'] =  \
np.log(action_type['action_sum']/action_all)
action_type

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio
0,1,27.0,0.0,6.0,0.0,33.0,-14.324978
1,2,47.0,0.0,14.0,2.0,63.0,-13.678350
2,3,63.0,0.0,4.0,1.0,68.0,-13.601977
3,4,49.0,0.0,1.0,0.0,50.0,-13.909462
4,5,150.0,0.0,13.0,10.0,173.0,-12.668194
...,...,...,...,...,...,...,...
424165,424166,79.0,0.0,11.0,0.0,90.0,-13.321676
424166,424167,28.0,0.0,6.0,1.0,35.0,-14.266137
424167,424168,216.0,0.0,6.0,1.0,223.0,-12.414313
424168,424169,277.0,0.0,17.0,3.0,297.0,-12.127753


In [17]:
action_type.head()

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio
0,1,27.0,0.0,6.0,0.0,33.0,-14.324978
1,2,47.0,0.0,14.0,2.0,63.0,-13.678350
2,3,63.0,0.0,4.0,1.0,68.0,-13.601977
3,4,49.0,0.0,1.0,0.0,50.0,-13.909462
4,5,150.0,0.0,13.0,10.0,173.0,-12.668194


In [18]:
action_type.shape

(424170, 7)

In [19]:
(action_type['action_sum']!=0).any()

True

In [20]:
action_type['type_0_ratio'] = np.log(action_type['action_type_0']/action_type['action_sum'] + 0.1)
action_type['type_1_ratio'] = np.log(action_type['action_type_1']/action_type['action_sum'] + 0.1)
action_type['type_2_ratio'] = np.log(action_type['action_type_2']/action_type['action_sum'] + 0.1)
action_type['type_3_ratio'] = np.log(action_type['action_type_3']/action_type['action_sum'] + 0.1)

In [21]:
action_type.head()

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,type_1_ratio,type_2_ratio,type_3_ratio
0,1,27.0,0.0,6.0,0.0,33.0,-14.324978,-0.085360,-2.302585,-1.266493,-2.302585
1,2,47.0,0.0,14.0,2.0,63.0,-13.678350,-0.167198,-2.302585,-1.132514,-2.026879
2,3,63.0,0.0,4.0,1.0,68.0,-13.601977,0.026126,-2.302585,-1.839962,-2.165384
3,4,49.0,0.0,1.0,0.0,50.0,-13.909462,0.076961,-2.302585,-2.120264,-2.302585
4,5,150.0,0.0,13.0,10.0,173.0,-12.668194,-0.033503,-2.302585,-1.742144,-1.846405


In [22]:
action_type.shape

(424170, 11)

In [23]:
# trans_ratio_ckick 点击转化率
# trans_ratio_cart  加购转化率
# trans_ratio_favor 收藏转化率 
action_type['trans_ratio_ckick'] = np.log(action_type['action_type_2'] / (action_type['action_type_0'] + 0.1))
action_type['trans_ratio_cart' ] = np.log(action_type['action_type_2'] / (action_type['action_type_1'] + 0.1))
action_type['trans_ratio_favor'] = np.log(action_type['action_type_2'] / (action_type['action_type_3'] + 0.1))
action_type

,user_id,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,type_1_ratio,type_2_ratio,type_3_ratio,trans_ratio_ckick,trans_ratio_cart,trans_ratio_favor
0,1,27.0,0.0,6.0,0.0,33.0,-14.324978,-0.085360,-2.302585,-1.266493,-2.302585,-1.507774,4.094345,4.094345
1,2,47.0,0.0,14.0,2.0,63.0,-13.678350,-0.167198,-2.302585,-1.132514,-2.026879,-1.213216,4.941642,1.897120
2,3,63.0,0.0,4.0,1.0,68.0,-13.601977,0.026126,-2.302585,-1.839962,-2.165384,-2.758426,3.688879,1.290984
3,4,49.0,0.0,1.0,0.0,50.0,-13.909462,0.076961,-2.302585,-2.120264,-2.302585,-3.893859,2.302585,2.302585
4,5,150.0,0.0,13.0,10.0,173.0,-12.668194,-0.033503,-2.302585,-1.742144,-1.846405,-2.446352,4.867534,0.252414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424165,424166,79.0,0.0,11.0,0.0,90.0,-13.321676,-0.022473,-2.302585,-1.504077,-2.302585,-1.972818,4.700480,4.700480
424166,424167,28.0,0.0,6.0,1.0,35.0,-14.266137,-0.105361,-2.302585,-1.304056,-2.051271,-1.544010,4.094345,1.696449
424167,424168,216.0,0.0,6.0,1.0,223.0,-12.414313,0.066359,-2.302585,-2.064310,-2.258718,-3.583982,4.094345,1.696449
424168,424169,277.0,0.0,17.0,3.0,297.0,-12.127753,0.032138,-2.302585,-1.849988,-2.206357,-2.791165,5.135798,1.701811


In [24]:
action_type.shape

(424170, 14)

# 用户--商铺交互特征

In [25]:
temp = pd.get_dummies(ul['action_type'],
                      prefix ="us_action_type")
action_seller = pd.concat([ul[['user_id','seller_id']],temp],
                          axis = 1)
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3
23288890,1,2245,1,0,0,0
23288891,1,2245,1,0,0,0
23288892,1,2245,1,0,0,0
23288893,1,2245,1,0,0,0
23288894,1,2245,1,0,0,0
...,...,...,...,...,...,...
13710705,424170,3736,1,0,0,0
13710706,424170,525,1,0,0,0
13710707,424170,3736,1,0,0,0
13710714,424170,4268,0,0,1,0


In [26]:
action_seller.shape

(54925330, 6)

In [27]:
action_seller = pd.concat([ul[['user_id','seller_id']],temp],
                          axis = 1)

In [28]:
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3
23288890,1,2245,1,0,0,0
23288891,1,2245,1,0,0,0
23288892,1,2245,1,0,0,0
23288893,1,2245,1,0,0,0
23288894,1,2245,1,0,0,0
...,...,...,...,...,...,...
13710705,424170,3736,1,0,0,0
13710706,424170,525,1,0,0,0
13710707,424170,3736,1,0,0,0
13710714,424170,4268,0,0,1,0


In [29]:
action_seller = action_seller.groupby(by = ['user_id','seller_id'],
                                  as_index = False).sum()

In [30]:
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3
0,1,471,1.0,0.0,0.0,0.0
1,1,739,1.0,0.0,0.0,0.0
2,1,925,3.0,0.0,1.0,0.0
3,1,1019,10.0,0.0,4.0,0.0
4,1,1156,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
14058661,424170,1082,1.0,0.0,0.0,0.0
14058662,424170,3469,1.0,0.0,0.0,0.0
14058663,424170,3736,10.0,0.0,0.0,0.0
14058664,424170,4268,24.0,0.0,1.0,0.0


In [31]:
action_seller['us_action_sum'] = action_seller.iloc[:,2:] \
.sum(axis = 1)
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3,us_action_sum
0,1,471,1.0,0.0,0.0,0.0,1.0
1,1,739,1.0,0.0,0.0,0.0,1.0
2,1,925,3.0,0.0,1.0,0.0,4.0
3,1,1019,10.0,0.0,4.0,0.0,14.0
4,1,1156,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
14058661,424170,1082,1.0,0.0,0.0,0.0,1.0
14058662,424170,3469,1.0,0.0,0.0,0.0,1.0
14058663,424170,3736,10.0,0.0,0.0,0.0,10.0
14058664,424170,4268,24.0,0.0,1.0,0.0,25.0


In [32]:
action_seller.shape

(14058666, 7)

In [33]:
action_seller.columns

Index(['user_id', 'seller_id', 'us_action_type_0', 'us_action_type_1',
       'us_action_type_2', 'us_action_type_3', 'us_action_sum'],
      dtype='object')

In [34]:
action_seller['us_action_ratio'] = action_seller.groupby(by = ['user_id'])['us_action_sum'].transform(lambda x:x/x.sum())
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3,us_action_sum,us_action_ratio
0,1,471,1.0,0.0,0.0,0.0,1.0,0.030303
1,1,739,1.0,0.0,0.0,0.0,1.0,0.030303
2,1,925,3.0,0.0,1.0,0.0,4.0,0.121212
3,1,1019,10.0,0.0,4.0,0.0,14.0,0.424242
4,1,1156,1.0,0.0,0.0,0.0,1.0,0.030303
...,...,...,...,...,...,...,...,...
14058661,424170,1082,1.0,0.0,0.0,0.0,1.0,0.025000
14058662,424170,3469,1.0,0.0,0.0,0.0,1.0,0.025000
14058663,424170,3736,10.0,0.0,0.0,0.0,10.0,0.250000
14058664,424170,4268,24.0,0.0,1.0,0.0,25.0,0.625000


In [35]:
(action_seller['us_action_sum']!=0).any()

True

In [36]:
action_seller['us_type_0_ratio'] = np.log(action_seller['us_action_type_0']/action_seller['us_action_sum'] + 0.1)
action_seller['us_type_1_ratio'] = np.log(action_seller['us_action_type_1']/action_seller['us_action_sum'] + 0.1)
action_seller['us_type_2_ratio'] = np.log(action_seller['us_action_type_2']/action_seller['us_action_sum'] + 0.1)
action_seller['us_type_3_ratio'] = np.log(action_seller['us_action_type_3']/action_seller['us_action_sum'] + 0.1)

In [37]:
action_seller['us_trans_ratio_ckick'] = np.log(action_seller['us_action_type_2'] / (action_seller['us_action_type_0'] + 0.1) + 0.1)
action_seller['us_trans_ratio_cart' ] = np.log(action_seller['us_action_type_2'] / (action_seller['us_action_type_1'] + 0.1) + 0.1)
action_seller['us_trans_ratio_favor'] = np.log(action_seller['us_action_type_2'] / (action_seller['us_action_type_3'] + 0.1) + 0.1)
action_seller

,user_id,seller_id,us_action_type_0,us_action_type_1,us_action_type_2,us_action_type_3,us_action_sum,us_action_ratio,us_type_0_ratio,us_type_1_ratio,us_type_2_ratio,us_type_3_ratio,us_trans_ratio_ckick,us_trans_ratio_cart,us_trans_ratio_favor
0,1,471,1.0,0.0,0.0,0.0,1.0,0.030303,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
1,1,739,1.0,0.0,0.0,0.0,1.0,0.030303,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
2,1,925,3.0,0.0,1.0,0.0,4.0,0.121212,-0.162519,-2.302585,-1.049822,-2.302585,-0.861375,2.312535,2.312535
3,1,1019,10.0,0.0,4.0,0.0,14.0,0.424242,-0.205444,-2.302585,-0.952658,-2.302585,-0.701100,3.691376,3.691376
4,1,1156,1.0,0.0,0.0,0.0,1.0,0.030303,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14058661,424170,1082,1.0,0.0,0.0,0.0,1.0,0.025000,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
14058662,424170,3469,1.0,0.0,0.0,0.0,1.0,0.025000,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
14058663,424170,3736,10.0,0.0,0.0,0.0,10.0,0.250000,0.095310,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585
14058664,424170,4268,24.0,0.0,1.0,0.0,25.0,0.625000,0.058269,-2.302585,-1.966113,-2.302585,-1.955500,2.312535,2.312535


In [38]:
action_seller.shape

(14058666, 15)

In [39]:
action_type.dtypes

user_id                int64
action_type_0        float64
action_type_1        float64
action_type_2        float64
action_type_3        float64
action_sum           float64
action_ratio         float64
type_0_ratio         float64
type_1_ratio         float64
type_2_ratio         float64
type_3_ratio         float64
trans_ratio_ckick    float64
trans_ratio_cart     float64
trans_ratio_favor    float64
dtype: object

In [40]:
action_seller.dtypes

user_id                   int64
seller_id                 int64
us_action_type_0        float64
us_action_type_1        float64
us_action_type_2        float64
us_action_type_3        float64
us_action_sum           float64
us_action_ratio         float64
us_type_0_ratio         float64
us_type_1_ratio         float64
us_type_2_ratio         float64
us_type_3_ratio         float64
us_trans_ratio_ckick    float64
us_trans_ratio_cart     float64
us_trans_ratio_favor    float64
dtype: object

In [41]:
def memory(data):
    b = data.memory_usage().sum()
    print(f"压缩前的大小: {data.memory_usage().sum()}")
    types = ['int64','float64']
#     print(f"压缩前的类型: {types}")
    print(f"压缩后的类型: {list(data.dtypes)}")
    for col in data.columns:
        c_type = data[col].dtypes
#         print(c_type)
#         print("______________")
        if c_type in types:
            cmax = data[col].max()
            cmin = data[col].min()
#             print(cmax,cmin)
#             print("______________")
            if str(c_type)[:3] == "int":
#                 print(str(c_type)[:3])
#                 print("_______________")
                if cmax < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif cmax < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif cmax < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int32)
                elif cmax < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int64)
            else:
                if cmax < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif cmax < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                elif cmax < np.finfo(np.float64).max:
                    data[col] = data[col].astype(np.float64)
    a = data.memory_usage().sum()
    ccc = list(data.dtypes)
    print("_______________________________________________")
    print(f"压缩后的大小: {data.memory_usage().sum()}")
    print(f"压缩比: {(b-a)/b*100}")
    print(f"压缩后的类型: {ccc}")
    print("")
    print("")

In [42]:
[memory(c) for c in [action_type,action_seller]]

压缩前的大小: 50900400
压缩后的类型: [dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')]
_______________________________________________
压缩后的大小: 17815140
压缩比: 65.0
压缩后的类型: [dtype('int64'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16')]


压缩前的大小: 1799509248
压缩后的类型: [dtype('int64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')]
_______________________________________________
压缩后的大小: 618581304
压缩比: 65.625
压缩后的类型: [dtype('

[None, None]

In [43]:
pickle.dump(action_type,open('action_type.pkl','wb'))
pickle.dump(action_seller,open('action_seller.pkl','wb'))

In [44]:
del action_seller,action_type
import gc
gc.collect()

513

In [45]:
action_type = pickle.load(open('action_type.pkl','rb'))
action_seller = pickle.load(open('action_seller.pkl','rb'))

In [46]:
lamb = lambda x:" ".join([str(i) for i in x])

In [47]:
agg_dict = {
    "item_id":lamb,
    "cat_id":lamb,
    "seller_id":lamb,
    "brand_id":lamb,
    "time_stamp":lamb,
    "action_type":lamb
}

In [48]:
name_dict = {
        "item_id":"all_item",
    "cat_id":"all_cat",
    "seller_id":"all_seller",
    "brand_id":"all_brand",
    "time_stamp":"all_time",
    "action_type":"all_action"
}

In [49]:
def agg_all(action_data,agg_dict,name_dict):
    action = action_data.groupby('user_id').agg(agg_dict).reset_index().rename(columns = name_dict)
    return action

In [50]:
data = agg_all(ul,agg_dict,name_dict)
data

,user_id,all_item,all_cat,all_seller,all_brand,all_time,all_action
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...,...
424165,424166,63042 82584 82584 1003360 1003360 641307 63042...,1075 1075 1075 1075 1075 1075 1075 1075 1075 1...,4844 4074 4074 4074 4074 3584 4844 4251 4251 4...,8080.0 3072.0 3072.0 3072.0 3072.0 7192.0 8080...,514 524 524 524 524 524 524 816 816 816 816 81...,2 2 0 2 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 0 0 0 ...
424166,424167,100537 100537 100537 100537 100537 100537 1005...,302 302 302 302 302 302 302 612 1429 825 825 1...,140 140 140 140 140 140 140 1583 1583 4766 476...,5856.0 5856.0 5856.0 5856.0 5856.0 5856.0 5856...,529 529 529 529 529 529 529 913 913 1021 1021 ...,0 0 0 2 0 0 0 0 0 0 2 2 0 0 0 0 0 0 3 0 0 0 2 ...
424167,424168,572710 161447 257300 385985 691733 820484 6196...,115 115 786 1290 1505 786 115 1505 184 455 407...,2783 923 4726 101 3049 2366 4661 3049 3185 411...,6316.0 5376.0 3188.0 1420.0 3768.0 5764.0 5172...,523 523 525 525 525 525 526 526 526 602 607 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
424168,424169,1076399 655072 385227 419195 385227 1050755 41...,464 812 351 351 351 351 351 351 351 351 351 35...,2228 3898 4731 4731 4731 4731 4731 4731 4731 4...,3452.0 6096.0 1387.0 1387.0 1387.0 1387.0 1387...,511 512 525 525 525 525 525 525 525 525 525 52...,2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [51]:
data.dtypes

user_id        int64
all_item      object
all_cat       object
all_seller    object
all_brand     object
all_time      object
all_action    object
dtype: object

In [52]:
[memory(c) for c in [data]]

压缩前的大小: 23753648
压缩后的类型: [dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')]
_______________________________________________
压缩后的大小: 23753648
压缩比: 0.0
压缩后的类型: [dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O')]




[None]

In [53]:
data.dtypes

user_id        int64
all_item      object
all_cat       object
all_seller    object
all_brand     object
all_time      object
all_action    object
dtype: object

In [54]:
pickle.dump(data,open('data.pkl','wb'))

In [55]:
data = pickle.load(open("data.pkl",'rb'))

In [56]:
from collections import Counter
l = [1,1,1,2,1,2,1,3,4,5,3,6]
Counter(l).most_common(3)

[(1, 5), (2, 2), (3, 2)]

In [57]:
from collections import Counter
l = [1,1,1,2,1,2,1,3,4,5,3,6]
Counter(l).most_common(3)[0][0]

1

In [58]:
lambda1 = lambda x:len(x.split(" "))
lambda2 = lambda x:set(x.split(" "))
lambda3 = lambda x:np.max([np.int(i) for i in x.split(" ")])
lambda4 = lambda x:np.min([np.int(i) for i in x.split(" ")])
lambda5 = lambda x:np.std([np.int(i) for i in x.split(" ")])
lambda6 = lambda x:Counter(x.split(" ")).most_common(3)[0][0]
lambda7 = lambda x:Counter(x.split(" ")).most_common(1)[0][0]
lambda8 = lambda x:Counter(x.split(" ")).most_common(1)[0][1]

In [59]:
data.columns

Index(['user_id', 'all_item', 'all_cat', 'all_seller', 'all_brand', 'all_time',
       'all_action'],
      dtype='object')

In [60]:
a = [i for i in data.columns if i not in ['user_id']]
a

['all_item', 'all_cat', 'all_seller', 'all_brand', 'all_time', 'all_action']

In [61]:
def user_a(data,aa):
    cc = "user_" + aa
    data[cc] = data[aa].apply(lambda1)
    return data

In [62]:
# def user_a(data,aa):
#     cc = "user_" + aa
#     dd = "set_" + aa
#     ee = "like_" + aa
#     ff = "like_count_" + aa
#     data[cc] = data[aa].apply(lambda1)
#     data[dd] = data[aa].apply(lambda2)
#     data[ee] = data[aa].apply(lambda7)
#     data[ff] = data[aa].apply(lambda8)
#     return data

In [63]:
data = user_a(data,"all_item")
data

,user_id,all_item,all_cat,all_seller,all_brand,all_time,all_action,user_all_item
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,33
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,63
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,68
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,50
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...,173
...,...,...,...,...,...,...,...,...
424165,424166,63042 82584 82584 1003360 1003360 641307 63042...,1075 1075 1075 1075 1075 1075 1075 1075 1075 1...,4844 4074 4074 4074 4074 3584 4844 4251 4251 4...,8080.0 3072.0 3072.0 3072.0 3072.0 7192.0 8080...,514 524 524 524 524 524 524 816 816 816 816 81...,2 2 0 2 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 0 0 0 ...,90
424166,424167,100537 100537 100537 100537 100537 100537 1005...,302 302 302 302 302 302 302 612 1429 825 825 1...,140 140 140 140 140 140 140 1583 1583 4766 476...,5856.0 5856.0 5856.0 5856.0 5856.0 5856.0 5856...,529 529 529 529 529 529 529 913 913 1021 1021 ...,0 0 0 2 0 0 0 0 0 0 2 2 0 0 0 0 0 0 3 0 0 0 2 ...,35
424167,424168,572710 161447 257300 385985 691733 820484 6196...,115 115 786 1290 1505 786 115 1505 184 455 407...,2783 923 4726 101 3049 2366 4661 3049 3185 411...,6316.0 5376.0 3188.0 1420.0 3768.0 5764.0 5172...,523 523 525 525 525 525 526 526 526 602 607 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,223
424168,424169,1076399 655072 385227 419195 385227 1050755 41...,464 812 351 351 351 351 351 351 351 351 351 35...,2228 3898 4731 4731 4731 4731 4731 4731 4731 4...,3452.0 6096.0 1387.0 1387.0 1387.0 1387.0 1387...,511 512 525 525 525 525 525 525 525 525 525 52...,2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 ...,297


In [64]:
data.shape

(424170, 8)

In [65]:
data.dtypes

user_id           int64
all_item         object
all_cat          object
all_seller       object
all_brand        object
all_time         object
all_action       object
user_all_item     int64
dtype: object

In [66]:
def unique_n(x):
    try:
        return len(set(x.split(" ")))
    except:
        return -1

In [67]:
cols = a
def user_a_unique(data,target_cols):
    for col in target_cols:
        data["{}_unique".format(col)] = data[col].apply(unique_n)
    return data
data = user_a_unique(data,cols)
data

,user_id,all_item,all_cat,all_seller,all_brand,all_time,all_action,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,33,12,6,9,9,5,2
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,63,43,14,14,15,9,3
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,68,45,19,23,21,13,3
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,50,28,13,12,12,10,2
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...,173,87,40,56,60,30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424165,424166,63042 82584 82584 1003360 1003360 641307 63042...,1075 1075 1075 1075 1075 1075 1075 1075 1075 1...,4844 4074 4074 4074 4074 3584 4844 4251 4251 4...,8080.0 3072.0 3072.0 3072.0 3072.0 7192.0 8080...,514 524 524 524 524 524 524 816 816 816 816 81...,2 2 0 2 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 0 0 0 ...,90,48,12,22,23,12,2
424166,424167,100537 100537 100537 100537 100537 100537 1005...,302 302 302 302 302 302 302 612 1429 825 825 1...,140 140 140 140 140 140 140 1583 1583 4766 476...,5856.0 5856.0 5856.0 5856.0 5856.0 5856.0 5856...,529 529 529 529 529 529 529 913 913 1021 1021 ...,0 0 0 2 0 0 0 0 0 0 2 2 0 0 0 0 0 0 3 0 0 0 2 ...,35,15,9,12,9,8,3
424167,424168,572710 161447 257300 385985 691733 820484 6196...,115 115 786 1290 1505 786 115 1505 184 455 407...,2783 923 4726 101 3049 2366 4661 3049 3185 411...,6316.0 5376.0 3188.0 1420.0 3768.0 5764.0 5172...,523 523 525 525 525 525 526 526 526 602 607 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,223,160,48,112,96,55,3
424168,424169,1076399 655072 385227 419195 385227 1050755 41...,464 812 351 351 351 351 351 351 351 351 351 35...,2228 3898 4731 4731 4731 4731 4731 4731 4731 4...,3452.0 6096.0 1387.0 1387.0 1387.0 1387.0 1387...,511 512 525 525 525 525 525 525 525 525 525 52...,2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 ...,297,176,46,102,100,34,3


In [68]:
data.shape

(424170, 14)

In [69]:
data['time_active'] = data['all_time'].apply(lambda3) - data['all_time'].apply(lambda4)

In [70]:
data.head()

,user_id,all_item,all_cat,all_seller,all_brand,all_time,all_action,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,33,12,6,9,9,5,2,102
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,63,43,14,14,15,9,3,584
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,68,45,19,23,21,13,3,595
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,50,28,13,12,12,10,2,584
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...,173,87,40,56,60,30,3,592


In [71]:
data.dtypes

user_id               int64
all_item             object
all_cat              object
all_seller           object
all_brand            object
all_time             object
all_action           object
user_all_item         int64
all_item_unique       int64
all_cat_unique        int64
all_seller_unique     int64
all_brand_unique      int64
all_time_unique       int64
all_action_unique     int64
time_active           int64
dtype: object

In [72]:
[memory(c) for c in [data]]

压缩前的大小: 50900528
压缩后的类型: [dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64')]
_______________________________________________
压缩后的大小: 30116198
压缩比: 40.83323064939523
压缩后的类型: [dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('int16'), dtype('int16'), dtype('int16'), dtype('int16'), dtype('int16'), dtype('int16'), dtype('int8'), dtype('int16')]




[None]

In [73]:
pickle.dump(data,open("data_z.pkl","wb"))

In [74]:
data_z = pickle.load(open("data_z.pkl","rb"))
data_z

,user_id,all_item,all_cat,all_seller,all_brand,all_time,all_action,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,33,12,6,9,9,5,2,102
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,63,43,14,14,15,9,3,584
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,68,45,19,23,21,13,3,595
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,50,28,13,12,12,10,2,584
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...,173,87,40,56,60,30,3,592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424165,424166,63042 82584 82584 1003360 1003360 641307 63042...,1075 1075 1075 1075 1075 1075 1075 1075 1075 1...,4844 4074 4074 4074 4074 3584 4844 4251 4251 4...,8080.0 3072.0 3072.0 3072.0 3072.0 7192.0 8080...,514 524 524 524 524 524 524 816 816 816 816 81...,2 2 0 2 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 0 0 0 ...,90,48,12,22,23,12,2,597
424166,424167,100537 100537 100537 100537 100537 100537 1005...,302 302 302 302 302 302 302 612 1429 825 825 1...,140 140 140 140 140 140 140 1583 1583 4766 476...,5856.0 5856.0 5856.0 5856.0 5856.0 5856.0 5856...,529 529 529 529 529 529 529 913 913 1021 1021 ...,0 0 0 2 0 0 0 0 0 0 2 2 0 0 0 0 0 0 3 0 0 0 2 ...,35,15,9,12,9,8,3,582
424167,424168,572710 161447 257300 385985 691733 820484 6196...,115 115 786 1290 1505 786 115 1505 184 455 407...,2783 923 4726 101 3049 2366 4661 3049 3185 411...,6316.0 5376.0 3188.0 1420.0 3768.0 5764.0 5172...,523 523 525 525 525 525 526 526 526 602 607 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,223,160,48,112,96,55,3,588
424168,424169,1076399 655072 385227 419195 385227 1050755 41...,464 812 351 351 351 351 351 351 351 351 351 35...,2228 3898 4731 4731 4731 4731 4731 4731 4731 4...,3452.0 6096.0 1387.0 1387.0 1387.0 1387.0 1387...,511 512 525 525 525 525 525 525 525 525 525 52...,2 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 ...,297,176,46,102,100,34,3,600


In [85]:
data_zz = data_z.iloc[:,7:]
data_zzz = data_z.iloc[:,[0]]
datas = pd.concat([data_zzz,data_zz],axis = 1)
datas

,user_id,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,1,33,12,6,9,9,5,2,102
1,2,63,43,14,14,15,9,3,584
2,3,68,45,19,23,21,13,3,595
3,4,50,28,13,12,12,10,2,584
4,5,173,87,40,56,60,30,3,592
...,...,...,...,...,...,...,...,...,...
424165,424166,90,48,12,22,23,12,2,597
424166,424167,35,15,9,12,9,8,3,582
424167,424168,223,160,48,112,96,55,3,588
424168,424169,297,176,46,102,100,34,3,600
